# 輸入關鍵字
* 輸入想要查找的關鍵字

In [103]:
keyword = input("Please enter the keyword:")
site = 'www.ettoday.net'
# site = input("Which media you want to access: ")

Please enter the keyword:防疫概念股


# import module

In [110]:
import requests
from bs4 import BeautifulSoup
import sys
import codecs
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
import re
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values': {
        'images': 2,
        'javascript': 2
    }
}
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome('./chromedriver', chrome_options=options)
# 開啟google網頁
driver.get("https://www.google.com")
driver.implicitly_wait(10)
# 輸入關鍵字、檢索
driver.find_element_by_xpath("""//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input""").click()
driver.implicitly_wait(1)

driver.find_element_by_xpath("""//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input""").clear()
driver.find_element_by_xpath("""//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input""").send_keys(keyword+' site:'+site)
driver.find_element_by_xpath("""//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input""").send_keys(Keys.ENTER)
driver.implicitly_wait(5)
# 進階搜尋：keyword、特定網域、特定時間
driver.find_element_by_xpath("""//*[@id="st-toggle"]""").click()  # 進階搜索
# 輸入關鍵字
driver.find_element_by_xpath("""//*[@id="xX4UFf"]""").click()
driver.find_element_by_xpath("""//*[@id="xX4UFf"]""").send_keys(keyword)
# 輸入網域
driver.find_element_by_xpath("""//*[@id="NqEkZd"]""").click()
driver.find_element_by_xpath("""//*[@id="NqEkZd"]""").send_keys(site)
# 輸入時間
driver.find_element_by_xpath("""/html/body/div[1]/div[5]/form/div[5]/div[3]/div[2]/div/select""").click()
driver.find_element_by_xpath("""/html/body/div[1]/div[5]/form/div[5]/div[3]/div[2]/div/select/option[5]""").click()  # option[4]是最近一個月內
### 若將上面option[4]改為5將可搜得最近1年的資料
driver.find_element_by_xpath("""/html/body/div[1]/div[5]/form/div[5]/div[9]/div[2]/input""").click()

/Users/jack1563311/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


# 自動化流程之迴圈
* 請輸入檔案名稱(要加.csv)
* 不需另外新建檔案，此迴圈會自動設立csv檔並將內容寫入csv檔中
* 過程會需要一點時間，請耐心等候


In [111]:
location = input("請(在您當前的資料夾中)輸入檔案儲存名稱(+.csv)：")
if os.path.exists(location):
    df = pd.read_csv(location, header=0)
else:
    df = pd.DataFrame(columns=['title', 'original_text', 'time'])

for j in range(50):  # 跑十頁
    soup = BeautifulSoup(driver.page_source, "html.parser")
    links = soup.select('.kCrYT')  # 只select html的原始碼
    count = 0  # 迴圈計數器
    news = []  # 
    for link in links:
        count += 1
        try:
            news.append(link.a['href'])
        except:
            continue
    L = []
    for i in range(len(news)):  # 將單一頁數的所有新聞裝進L
        try:
            r = requests.get(news[i][7:])
            news_url = news[i][7:]
        except:
            r = requests.get(news[i])
            news_url = news[i]
        news_try = requests.get(news_url)
        soup = BeautifulSoup(news_try.text, 'html.parser')
        try:
            titles = soup.h1.text
            stories = soup.select(".story")
            L.append([])
            L[i].append(titles)
            # 先對文字做預處理，去除換行以及回到開頭(\n,\r)
            pattern = '[\u4e00-\u9fa5]+'
            words = re.findall(pattern, stories[0].text.replace('\r', '').replace('\n', ''))
            sentence = ''
            for word in words:
                sentence += word
            L[i].append(sentence)
            L[i].append(soup.time.text[61:78])
            singlepage_df = pd.DataFrame(L, columns=['title', 'original_text', 'time'])
        except:
            continue
    df = df.append(singlepage_df, ignore_index = True)
    
    if j == 0:
        driver.find_element_by_xpath("""//*[@id="main"]/footer/div[1]/div/div/a""").click()
    else:
        try:
            try:
                driver.find_element_by_xpath(f"""//*[@id="main"]/footer/div[1]/div/div/a[{j + 1}]/span""").click()
            except:
                driver.find_element_by_xpath(f"""//*[@id="main"]/footer/div[1]/div/div/a[3]/span""").click()
        except:
            break
    driver.implicitly_wait(5)

droplist = []
count = 0
for i in range(len(df)):
    try:
        if int(df['time'][i][:4]) < 2020:
            droplist.append(i)
    except:
        print('該則新聞無法抓取：'+str(count))
    count+=1
    
csvFile = open(location, 'a' ,encoding='utf-8')
df.drop(droplist).to_csv(csvFile, mode = 'a', columns=['title', 'original_text', 'time'], encoding='utf-8')
print("資料已存取")

請(在您當前的資料夾中)輸入檔案儲存名稱(+.csv)：防疫概念股(re).csv
該則新聞無法抓取時間：265
該則新聞無法抓取時間：275
資料已存取
